In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [2]:
from collections import Counter

import scipy.sparse as spr
import pickle

import import_ipynb
from util import write_json, load_json
from evaluate import CustomEvaluator
from ubcf_model import rec

importing Jupyter notebook from util.ipynb
importing Jupyter notebook from evaluate.ipynb
importing Jupyter notebook from ubcf_model.ipynb


In [65]:
import pandas as pd

from load_data import train_playlist
Ratings = load_json("./data/processedData/rating.json")
Ratings = pd.DataFrame(eval(Ratings))
Ratings['size'] = 0

playlist_size = Ratings.groupby(['playlist_id'],as_index=True)['rating'].count()

In [117]:
from load_data import carol_data
carol_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,"[GN1601, GN1615, GN1610]",20171006,Christmas at Westminster Abbey,10104201,"[345328, 36652, 347399]",Anonymous: Up! Awake! From Highest Steeple - A...,[GN1600],"[Choir Of Westminster Abbey, Simon Preston, Ch...",23
1,"[GN1701, GN1703, GN1702]",20121109,Christmas (Deluxe Special Edition),2167838,[35370],Mis Deseos \/ Feliz Navidad (with Thalia),[GN1700],[Michael Buble],41
2,[GN0101],20190408,있어줄래,10271322,[500421],있어줄래,[GN0100],[길구봉구],54
3,"[GN0105, GN0101]",20121204,WINTER POETRY,2169041,[1741],너에게 전하지 못한 말,[GN0100],[신혜성],70
4,"[GN1304, GN1301]",20110810,Simple Steps,2058451,[614091],Rollercoaster,[GN1300],[Sam Ock],71


In [86]:
for playlist in Ratings.index:
    Ratings['size'][playlist] = playlist_size[Ratings['playlist_id'][playlist]]

In [88]:
Ratings['importance'] = Ratings['rating'] / Ratings['size']

In [91]:
Ratings.head()

,playlist_id,song_id,rating,size,importance
0,107263,212655,1,30,0.033333
1,107263,483486,1,30,0.033333
2,107263,10139,1,30,0.033333
3,107263,452481,1,30,0.033333
4,107263,328894,1,30,0.033333


In [92]:
check = pd.pivot_table(Ratings,values='rating',index='playlist_id',columns='song_id')
check.head()

song_id,23,54,70,71,140,149,250,289,311,416,...,707581,707603,707679,707707,707792,707796,707821,707913,707969,707970
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
final = pd.pivot_table(Ratings,values='importance',index='playlist_id',columns='song_id')
final.head()

song_id,23,54,70,71,140,149,250,289,311,416,...,707581,707603,707679,707707,707792,707796,707821,707913,707969,707970
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# Replacing NaN by Movie Average
final_song = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_plylst = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [95]:
final_song.head()

song_id,23,54,70,71,140,149,250,289,311,416,...,707581,707603,707679,707707,707792,707796,707821,707913,707969,707970
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,0.033333,0.017544,0.03125,0.005,0.005,0.047619,0.027985,0.017857,0.033333,0.055556,...,0.043478,0.005,0.033144,0.028571,0.020833,0.043478,0.014286,0.020833,0.016581,0.005
171,0.033333,0.017544,0.03125,0.005,0.005,0.047619,0.027985,0.017857,0.033333,0.055556,...,0.043478,0.005,0.033144,0.028571,0.020833,0.043478,0.014286,0.020833,0.016581,0.005
237,0.033333,0.017544,0.03125,0.005,0.005,0.047619,0.027985,0.017857,0.033333,0.055556,...,0.043478,0.005,0.033144,0.028571,0.020833,0.043478,0.014286,0.020833,0.016581,0.005
941,0.033333,0.017544,0.03125,0.005,0.005,0.047619,0.027985,0.017857,0.033333,0.055556,...,0.043478,0.005,0.033144,0.028571,0.020833,0.043478,0.014286,0.020833,0.016581,0.005
1035,0.033333,0.017544,0.03125,0.005,0.005,0.047619,0.027985,0.017857,0.033333,0.055556,...,0.043478,0.005,0.033144,0.028571,0.020833,0.043478,0.014286,0.020833,0.016581,0.005


In [96]:
final_plylst.head()

song_id,23,54,70,71,140,149,250,289,311,416,...,707581,707603,707679,707707,707792,707796,707821,707913,707969,707970
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
171,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727
237,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,...,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632
941,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1035,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [97]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_plylst)
np.fill_diagonal(b, 0 )
similarity_with_plylst = pd.DataFrame(b,index=final_plylst.index)
similarity_with_plylst.columns=final_plylst.index
similarity_with_plylst.head()

playlist_id,156,171,237,941,1035,1096,1285,1440,1608,1625,...,152523,152572,152628,152684,152767,152874,152905,153136,153177,153196
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
171,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
237,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
941,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1035,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [99]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_song)
np.fill_diagonal(cosine, 0 )
similarity_with_song = pd.DataFrame(cosine,index=final_song.index)
similarity_with_song.columns=final_plylst.index
similarity_with_song.head()

playlist_id,156,171,237,941,1035,1096,1285,1440,1608,1625,...,152523,152572,152628,152684,152767,152874,152905,153136,153177,153196
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,0.000000,0.999854,0.999800,1.000000,1.000000,1.000000,1.000000,0.999860,0.998836,0.999914,...,0.996145,0.999662,0.999916,0.999549,0.999739,0.999924,1.000000,1.000000,0.999552,0.999635
171,0.999854,0.000000,0.999657,0.999854,0.999854,0.999854,0.999854,0.999709,0.998797,0.999795,...,0.996003,0.999599,0.999772,0.999356,0.999596,0.999780,0.999854,0.999854,0.999409,0.999478
237,0.999800,0.999657,0.000000,0.999800,0.999800,0.999800,0.999800,0.999662,0.998597,0.999716,...,0.995947,0.999466,0.999718,0.999351,0.999541,0.999727,0.999800,0.999800,0.999354,0.999438
941,1.000000,0.999854,0.999800,0.000000,1.000000,1.000000,1.000000,0.999860,0.998836,0.999914,...,0.996145,0.999662,0.999916,0.999549,0.999739,0.999924,1.000000,1.000000,0.999552,0.999635
1035,1.000000,0.999854,0.999800,1.000000,0.000000,1.000000,1.000000,0.999860,0.998836,0.999914,...,0.996145,0.999662,0.999916,0.999549,0.999739,0.999924,1.000000,1.000000,0.999552,0.999635


In [100]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [101]:
# top 30 neighbours for each user
sim_user_30_p = find_n_neighbours(similarity_with_plylst,30)
sim_user_30_p.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,91299,70525,25803,113189,14090,148417,151080,91160,32078,74386,...,49712,31700,33806,40379,138851,138753,125651,143745,49836,1950
171,91299,25803,70525,14090,113189,148417,91160,151080,32078,132647,...,79406,108340,53775,49836,31700,49712,1950,138851,40379,138753
237,91299,25803,70525,14090,113189,148417,91160,151080,101201,52107,...,79406,108340,32078,31700,49712,1950,138753,138851,33806,49836
941,91299,70525,25803,113189,14090,148417,151080,91160,32078,74386,...,49712,31700,33806,40379,138851,138753,125651,143745,49836,1950
1035,91299,70525,25803,113189,14090,148417,151080,91160,32078,74386,...,49712,31700,33806,40379,138851,138753,125651,143745,49836,1950


In [102]:
# top 30 neighbours for each user
sim_user_30_s = find_n_neighbours(similarity_with_song,30)
sim_user_30_s.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
playlist_id,,,,,,,,,,,,,,,,,,,,,
156,78157,42483,90538,90759,119554,43856,43855,119498,119229,41850,...,144666,120188,50105,135228,120275,49630,48799,135421,135441,48284
171,37281,111737,140253,136979,114791,120742,113852,72126,35414,120109,...,34802,20528,69868,69460,69386,70501,71388,69067,71592,69052
237,126164,120742,140253,127756,6650,111737,72126,113852,136979,41413,...,72423,34802,20528,72387,68582,72377,69460,68718,69868,68925
941,156,69386,73634,73435,72387,72377,71592,71388,70501,69868,...,64367,63888,74315,74618,74688,75310,90390,90043,89385,88636
1035,156,69386,73634,73435,72387,72377,71592,71388,70501,69868,...,64367,63888,74315,74618,74688,75310,90390,90043,89385,88636


In [104]:
def get_playlist_similar_songs( py1, py2 ):
    common_songs = Ratings[Ratings.playlist_id == py1].merge(
    Ratings[Ratings.playlist_id == py2],
    on = "song_id",
    how = "inner" )
    return common_songs.merge( movies, on = 'song_id' )

In [112]:
def User_item_score(user,item):
    a = sim_user_30_s[sim_user_30_s.index==user].values
    b = a.squeeze().tolist()
    c = final_song.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Ratings.loc[Ratings['playlist_id'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_song.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [113]:
score = User_item_score(156,23)
print("score (u,i) is",score)

score (u,i) is 1.0333333333333334


In [114]:
Ratings = Ratings.astype({"song_id": str})
Song_plylst = Ratings.groupby(by = 'playlist_id')['song_id'].apply(lambda x:','.join(x))

In [143]:
def User_item_score1(user):
    song_seen_by_playlist = check.columns[check[check.index == user].notna().any()].tolist()
    a = sim_user_30_s[sim_user_30_s.index==user].values
    b = a.squeeze().tolist()
    d = Song_plylst[Song_plylst.index.isin(b)]
    l = ','.join(d.values)
    song_seen_by_similar_playlists = l.split(',')
    Songs_under_consideration = list(set(song_seen_by_similar_playlists) - set(list(map(str, song_seen_by_playlist))))
    Songs_under_consideration = list(map(int, Songs_under_consideration))
    score = []
    for item in Songs_under_consideration:
        c = final_song.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Ratings.loc[Ratings['playlist_id'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_song.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'id':Songs_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Song_Name = top_5_recommendation.merge(carol_data, how='inner', on='id')
    Song_Names = Song_Name.song_name.values.tolist()
    return score

In [144]:
user = int(input("Enter the playlist id to whom you want to recommend : "))
predicted_songs = User_item_score1(user)
print(" ")
#print("The Recommendations for playlist Id : 370")
print("   ")
print(predicted_songs)
#for i in predicted_songs:
#    print(i)

Enter the playlist id to whom you want to recommend : 156
 
   
[]
